In [ ]:
import random
import numpy as np
import pandas as pd
import json
from urllib.request import urlopen
import osmnx as ox
import networkx as nx
from munkres import Munkres, print_matrix

stationfile = "C:\\Users\\Su_hy\\Documents\\study\\Research\\battery-swapping\\data\\station_Beijing.csv"

In [ ]:
# import driving map of Beijing
G = ox.graph_from_place("北京市,中国", network_type="drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
# impute speed on all edges missing data
G = ox.add_edge_speeds(G)
# calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)

In [ ]:
TIMEINTERVAL = 10  # 每10min统计
FULLBATTERY = 99.9  # 电池总量
MINBATTERY = 20  # 需要进行换点的剩余电池量
TESTTIME = 1000  # 总分钟数
BATTERYCHARGING = 60  # 将一个空电池充满需要的时间(min)
CHARGINGTIME = 45
EVSWAPPING = 5  # 换一个电池需要的时间(min)
EVNUM = 500  # 几辆车
BAttERYNUM=30 # 换电站的电池量
STATIONNUM=20 #换电站20+充电站0


# 一些weight计算
distanceSum = 0
timeSum = 0
waitSum = 0
chargeBattery = 0  # 新充了多少个电池
consumeBattery = 0  # 和EV换了多少个电池
batteryShort = 0
swappingSum = 0
swappingAverage = 0
chargingSum = 0

drivingEV = [[] for i in range(EVNUM - 1)]  # 有几行
waitingEV = []
chargingEV = [[] for i in range(STATIONNUM)]

In [ ]:
w1 = 1
w2 = 1.5
w3 = 1.2
w4 = 3

In [ ]:
# 事先的站点打分数据等
Pre_Rating = []
for i in range(STATIONNUM):
    Pre_Rating.append(5*random.random())

In [ ]:
nodes = gdf_nodes
nodes = nodes.reset_index()

station_df = pd.read_csv(stationfile,header=None)

# 初始设置车 & 电量
for i in range(0, EVNUM - 1):
    drivingEV[i].append(i + 1)
    drivingEV[i].append(random.uniform(MINBATTERY, FULLBATTERY))

drivingEV.sort(key=lambda drivingEV: drivingEV[1])  # 根据剩余电量排序
station_df

In [ ]:
def decideNode(loc):
    la = loc[0]
    lo = loc[1]
    os_node = nodes.iloc[((nodes['y'] - la).abs() + (nodes['x'] - lo).abs()).argsort()].iloc[0, 0]
    return os_node


def calculateTD(EVno, stationno):
    orig = EVno
    dest = stationno
    route1 = ox.shortest_path(G, orig, dest, weight="travel_time")
    # route2 = ox.shortest_path(G, orig, dest, weight="length")
    route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
    # route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
    route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
    # route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
    return [route1_length, route1_time]

# greedy matching
def matching(EVno):
    min_time = 9999999999999999
    minstation = station_df.iloc[0, 4]
    for i in range(0, 20):
        stationno = station_df.iloc[i, 4]
        cur = calculateTD(EVno, stationno)[1]
        dis = calculateTD(EVno, stationno)[0]
        if cur < min_time:
            min_time = cur
            min_distance = dis
            minstation = station_df.iloc[i, 0]
    matchResult = [minstation, min_time, min_distance]
    return matchResult


def assignLoc(tIndex, EVno):  # 读第x行的文件吧 随机分配
    filename = "C:\\Users\\Su_hy\\Documents\\study\\Research\\battery-swapping\\simulation_v1.0\\taxiGpsData\\T-drive Taxi Trajectories\\release\\taxi_log_2008_by_id\\%d.txt" % (
        EVno)
    dataset = pd.read_csv(filename, header=None, names=['no', 'time', 'lng', 'lat'])
    return [dataset.at[tIndex, 'lat'], dataset.at[tIndex, 'lng']]


def energy_cost(dis, speed):
    if 0 < speed <= 50:
        cost = (30 - speed / 4) * dis/100
    elif 100>= speed >=60 :
        cost = (17.5 + speed / 8) * dis/100
    elif 50<=speed<=60:
        cost = 18*dis/100
    else:
        cost=22.5*speed/100
    return cost

In [ ]:
def calCost(EV, station):
    EVno = decideNode(EV[2])
    station_no = station[4]
    cur = calculateTD(EVno, station_no)[1]
    dis = calculateTD(EVno, station_no)[0]
    rating = station.iloc[7]
    wait = 0
    if(station[9] == 0): # is a charging station TODO: 需要加上当前队列人数
        wait = CHARGINGTIME * len(chargingEV[station[0]])
    if(station[5] == 0):
        wait += BATTERYCHARGING * len(chargingEV[station[0]])
    cost = (w1*cur + w2*wait)*(w3 if station[9] else 1)+w4*(10-rating) # rating go from 5 to 10
    return cost

# def KMmatching(EVlist):
#     print("1")
#     battery_index=[]
#     y_length=0
#     for s in range(20): # calculate battery num
#         y_length+=station_df.iloc[s, 5]
#         battery_index.append(y_length)
#     costMatrix=np.zeros((y_length+STATIONNUM-20, y_length+STATIONNUM-20))
#     i=0
#     j=0
#     print("2")
#     for EV in EVlist:
#         print("EV: {}".format(i))
#         j = 0
#         if (i >= y_length+STATIONNUM-20): break
#         for s in range(STATIONNUM):
#             print("Station: {}".format(s))
#             cost = calCost(EV,station_df.iloc[s,:])
#             for k in range(station_df.iloc[s,5]):
#               costMatrix[i,j]=cost
#               j=j+1
#         i=i+1
#     print("3")
#     m = Munkres()
#     indexes_tuple = m.compute(costMatrix)
#     print("4")
#     indexes = [list(item) for item in indexes_tuple]
#     for i in range(len(EVlist)):
#         cur_index = indexes[i][1]
#         indexes[i][1]=battery_index.index(min(i for i in battery_index if i >= cur_index))
#     print("5")
    # return indexes
        

In [ ]:
def calculateCost(EVno, station, chargingEV):
    dis, cur = calculateTD(EVno, station[4])
    rating = station[7]
    wait = 0
    if(station[9] == 0): # is a charging station TODO: 需要加上当前队列人数
        wait = CHARGINGTIME * len(chargingEV)
    if(station[5] == 0):
        wait += BATTERYCHARGING * len(chargingEV)
    cost = (w1*cur + w2*wait)*(w3 if station[9] else 1)+w4*(10-rating) # rating go from 5 to 10
    return cost, dis, cur

def KMmatching(EVlist, chargingEV):
    y_length = station_df.iloc[:,5].sum()
    length = len(EVlist)
    costMatrix = np.zeros((length, y_length))
    costStorage = np.zeros((length, y_length), dtype='i,i,i')
    i = 0
    for EV in EVlist:
        j = 0
        for m in range(0, STATIONNUM):  # m is the station no
            cur_sta_cost, distance, time = calculateCost(decideNode(EV[2]), station_df.iloc[m, :].to_numpy(),
                                                         chargingEV[m])
            for k in range(0, station_df.iloc[m, 5]):  # station[0] is battery num, for charging num = 1
                costMatrix[i, j] = cur_sta_cost
                costStorage[i][j] = (distance, time, m)
                j = j + 1
        i = i + 1
    a = costMatrix.copy()
    m = Munkres()
    indexes = m.compute(a)
    return indexes, costStorage

In [ ]:
for tIndex in range(0, int(TESTTIME / TIMEINTERVAL)):  # 每个反馈周期
    print("{} round".format(tIndex))
    # 检查每个EV
    for EV in drivingEV:
        speed = random.uniform(1, 10) + random.choice([0, 10, 15, 15, 15, 15, 20])
        distance = TIMEINTERVAL * speed / 60
        EV[1] -= energy_cost(distance, speed)  # 期间消耗电量
        # 进入待匹配队列
        if EV[1] < MINBATTERY:
            loc = assignLoc(tIndex, EV[0])  # 随机地址
            EV.append(loc)
            EV.append(tIndex)
            waitingEV.append(EV)
            drivingEV.remove(EV)
        drivingEV.sort(key=lambda drivingEV: drivingEV[1]) 
    # 进行算法匹配
    matchResult = []
    if len(waitingEV) > 0:
        matchResult, costResult = KMmatching(waitingEV, chargingEV)
        # print("match done")
        # 匹配结果进入充电队列
        curCharging = [[] for i in range(STATIONNUM)]
        for match in matchResult:
            row = match[0]
            col = match[1]
            timecost = costResult[row][col][1]
            timeSum += timecost
            distanceSum += costResult[row][col][0]  # 到达这个充电站的消耗
            arrivingT = timecost / 60 + tIndex * TIMEINTERVAL  # 到达时间
            stationNo = costResult[row][col][2]
            curCharging[stationNo].append([waitingEV[row], arrivingT])
            curcurCharging = curCharging[stationNo]
            curcurCharging.sort(key=lambda curcurCharging: curCharging[1])
            curCharging[stationNo] = curcurCharging
            break
        for i in range(0, 20):
            chargingEV[i].extend(curCharging[i])
            break
        waitingEV = []
    # # 匈牙利算法
    # match_result = KMmatching(waitingEV)
    # for pair in match_result:
    #     if(pair[0] >= len(waitingEV)): break
    #     min_station_no = pair[1]
    #     EV_number = waitingEV[pair[0]][0]
    #     EV_no = decideNode(waitingEV[pair[0]][2])
    #     min_time = calculateTD(EV_no,station_df.iloc[min_station_no,4])[1]
    #     min_dis = calculateTD(EV_no,station_df.iloc[min_station_no,4])[0]
    #     match = [min_station_no, min_time, min_dis]
    #     matchResult.append(match)
    # greedy算法
    # for EV in waitingEV: 
    #     node_ev=decideNode(EV[2])
    #     match = matching(node_ev)
    #     match.append(EV)
    #     matchResult.append(match)
    # waitingEV = []
    # # 匹配结果进入充电队列
    # curCharging = [[] for i in range(STATIONNUM)]
    # for match in matchResult:
    #     print(match)
    #     timeSum += match[1]/60
    #     distanceSum += match[2]  # 到达这个充电站的消耗
    #     arrivingT = match[1] / 60 + tIndex * TIMEINTERVAL  # 到达时间
    #     curCharging[match[0]].append([match[3], arrivingT])
    #     curcurCharging = curCharging[match[0]]
    #     curcurCharging.sort(key=lambda curcurCharging: curCharging[1])
    #     curCharging[match[0]] = curcurCharging
    # for i in range(0, STATIONNUM):
    #     chargingEV[i].extend(curCharging[i])
    # print(chargingEV)
    # 对电池情况进行更新
    for i in range(0, STATIONNUM):
        if tIndex % (BATTERYCHARGING / TIMEINTERVAL) == 0 and station_df.iloc[i, 6] >= 1:
            chargeBattery += 1
            station_df.iloc[i,5] += 1
            station_df.iloc[i,6] -= 1
    # 对于充电队列进行审查（充电队列有先后顺序）
    for i in range(0, 20):
        for cEV in chargingEV[i]:
            if cEV[1] + EVSWAPPING <= tIndex * TIMEINTERVAL and station_df.iloc[i,5] >= 1:  # 可以换电
                consumeBattery += 1
                cEV[0][1] = FULLBATTERY
                station_df.iloc[i,5] -= 1 # 换电站电池-1
                station_df.iloc[i,6] += 1
                oldLeavingT = cEV[1] + EVSWAPPING  # 前一辆车离开换电站的时间
                swappingSum+=oldLeavingT-cEV[0][3]*10 # 计算这辆车的换电时间总和
                cEV[0]=cEV[0][0:2]
                drivingEV.append(cEV[0])
                chargingEV[i].remove(cEV)
                if len(chargingEV[i]) >= 1: # 等待前一个完成
                    curArrivingT = chargingEV[i][0][1]
                    chargingEV[i][0][1] = max(oldLeavingT, curArrivingT)
                    waitSum += max(0, oldLeavingT - curArrivingT)
                cur_rate = Pre_Rating[i] + random.random() - 0.5 # Rate在某个pre的基础上上下浮动
                station_df.iloc[i, 7] = (station_df.iloc[i, 7]*station_df.iloc[i, 8]+cur_rate)/(station_df.iloc[i, 8]+1)
                station_df.iloc[i, 8] += 1
            elif station_df.iloc[i,5] == 0:  # 没电池了
                cEV[1] = (tIndex + 1) * TIMEINTERVAL
                print("no more battery")
                batteryShort += 1
                break
            elif cEV[1] + EVSWAPPING > tIndex * TIMEINTERVAL:  # 还没到换电时间
                break
    for i in range(20, STATIONNUM): # 审查充电站
        for cEV in chargingEV[i]:
            if cEV[1] + CHARGINGTIME <= tIndex * TIMEINTERVAL:
                oldLeavingT = cEV[1] + CHARGINGTIME
                chargingSum += oldLeavingT-cEV[0][3]*10
                cEV[0]=cEV[0][0:2]
                drivingEV.append(cEV[0])
                chargingEV[i].remove(cEV)
                if len(chargingEV[i]) >= 1: # 等待前一个完成
                    curArrivingT = chargingEV[i][0][1]
                    chargingEV[i][0][1] = max(oldLeavingT, curArrivingT)
                    waitSum += max(0, oldLeavingT - curArrivingT)
                cur_rate = Pre_Rating[i] + random.random() - 0.5 # Rate在某个pre的基础上上下浮动
                station_df.iloc[i, 7] = (station_df.iloc[i, 7]*station_df.iloc[i, 8]+cur_rate)/(station_df.iloc[i, 8]+1)
                station_df.iloc[i, 8] += 1
            elif cEV[1] + EVSWAPPING > tIndex * TIMEINTERVAL:  # 还没到时间
                break

In [ ]:
# swappingAverage = swappingSum/consumeBattery

print("Total distance is %f meters" % (distanceSum))
# print("Average travle distance is %f meters" % (distanceSum/consumeBattery))
print("Total travel time is %f mins" % (timeSum))
print("Total waiting time is %f mins" % (waitSum))
print("Total consumption time of battery-swapping is %f mins" % (swappingSum))
# print("Average consumption time of battery-swapping is %f mins" % (swappingAverage))
print("Total newly charged battery is %f" % (chargeBattery))
print("Total consumed battery is %f" % (consumeBattery))
print("Number of cases where there's is short of batteries is %d" % (batteryShort))